# Requirement:
- numpy 1.19.5
- tensorflow 2.5.0
- tensorflow_privacy 0.6.1
- sklearn 0.24.2

Before performing the experiments, you need to fit the **pickle_file**(path for tabular QMNIST data) according to your own PC.

In [1]:
import numpy as np

import pickle
import tensorflow as tf
from tensorflow.keras import layers
import random as python_random

from sklearn.model_selection import train_test_split

# Load QMNIST data.

In [2]:
pickle_file = '/home/jiangnan/Desktop/ppml-workshop/data/QMNIST_tabular_ppml.pickle'

with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  x_defender = pickle_data['x_defender']
  x_reserve = pickle_data['x_reserve']
  y_defender = pickle_data['y_defender']
  y_reserve = pickle_data['y_reserve']
  del pickle_data
print('Data loaded.')

Data loaded.


In [3]:
NUM_CLASSES = 10

y_defender = y_defender[:,0]
y_reserve = y_reserve[:,0]

#y_defender = np.expand_dims(y_defender,axis=1)
#y_reserve = np.expand_dims(y_reserve,axis=1)

#y_defender = tf.keras.utils.to_categorical(y_defender, num_classes=NUM_CLASSES)
#y_reserve = tf.keras.utils.to_categorical(y_reserve, num_classes=NUM_CLASSES)

# Defender model $M_D$

import the defender model which need to be tested.

In [4]:
from sklearn.naive_bayes import GaussianNB

def defender_model_fn():
    """The architecture of the defender (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = GaussianNB()

    return model

## Train $M_D$ 

In [5]:
#epochs = 5
#batch_size = 64
#validation_split = 0.5

In [7]:
number_records = [48, 100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200, 102400, 200000]
number_records_used = number_records[0]

data_in = x_defender[:number_records_used], y_defender[:number_records_used]
data_out = x_reserve[:number_records_used], y_reserve[:number_records_used]

In [8]:
#defender_model_path = '/home/jiangnan/Desktop/ppml-workshop/defender_trained_models/simple_fn_48examples'
#defender_model = tf.keras.models.load_model(defender_model_path)

In [9]:
defender_model = defender_model_fn()
defender_model.fit(data_in[0],data_in[1])

GaussianNB()

## Evaluate the utility of $M_D$ on utility evaluation dataset $E_U$ (which is equal to reserve dataset $R$).

Evaluation metrics: **Accuracy** & **AUC**

In [10]:
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances

In [11]:
predict_reserve_proba = defender_model.predict_proba(data_out[0])
predict_reserve = np.argmax(predict_reserve_proba, axis=1)
acc = accuracy_score(data_out[1], predict_reserve)
#loss = log_loss(data_out[1], predict_reserve_proba)

In [12]:
auc_by_class = []

# compute auc per class then take the average value
for i in range(NUM_CLASSES):
  class_indices = data_out[1] == i
  if np.sum(class_indices) == 0:
        continue
  fpr, tpr, thresholds = metrics.roc_curve(class_indices, predict_reserve_proba[:,i])
  auc = metrics.auc(fpr, tpr)
  auc_by_class.append(auc)

average_auc = np.mean(auc_by_class)

In [13]:
print('Utility of defender model:')
print('Acc: {}'.format(acc))
print('Auc: {}'.format(average_auc))
#print('Loss: {}'.format(loss))

Utility of defender model:
Acc: 0.6666666666666666
Auc: 0.7689347200700154


In [14]:
utility = max(2*average_auc -1,0)
print('Defender model utility: {}'.format(utility))

Defender model utility: 0.5378694401400308


# Oracle attack model $M_A$

In [15]:
import random
from tqdm import tqdm

In [16]:
results = []

for i in tqdm(range(100)):
    
    random_index = random.randint(0,number_records[0]-1)

    evaluation_data_in = data_in[0][random_index]
    evaluation_label_in = data_in[1][random_index]

    evaluation_data_out = data_out[0][random_index]
    evaluation_label_out = data_out[1][random_index]

    evaluation_data = np.array([evaluation_data_in, evaluation_data_out])
    evaluation_label = np.array([evaluation_label_in, evaluation_label_out])

    evaluation = evaluation_data, evaluation_label

    
    attack_train_data_in = np.delete(data_in[0], random_index, axis=0)
    attack_train_label_in = np.delete(data_in[1], random_index, axis=0)

    attack_in = attack_train_data_in, attack_train_label_in

    
    attack_train_data_out = np.delete(data_out[0], random_index, axis=0)
    attack_train_label_out = np.delete(data_out[1], random_index, axis=0)

    attack_out = attack_train_data_out, attack_train_label_out
    

    predict = defender_model.predict_log_proba(attack_in[0])
    similarities = []
    
    for i in range(evaluation[0].shape[0]):
    
        attack_in_plus_one = np.vstack((evaluation[0][i].reshape(1,attack_in[0].shape[1]),attack_in[0])), np.hstack(( evaluation[1][i],attack_in[1]))

        M_cD = defender_model_fn()
        M_cD.fit(attack_in_plus_one[0], attack_in_plus_one[1])
        M_cD_predict = M_cD.predict_log_proba(attack_in[0])
        
        
        similarity = np.mean(np.linalg.norm(M_cD_predict-predict, axis=1))
        similarities.append(similarity)


    if similarities[0] < similarities[1]:
        results.append(1)
    else:
        results.append(0)

100%|██████████| 100/100 [00:00<00:00, 157.25it/s]


In [17]:
results[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [18]:
n = len(results)
p = 1-np.sum(results)/n

privacy = min(2*p,1)
variance = 2*p*(1-p)/n
sigma_error = 2*np.sqrt(p*(1-p)/n)

In [20]:
print('Defender model privacy: {}'.format(privacy))
print('Privacy variance: {}'.format(variance))
print('Privacy error: {}'.format(sigma_error))

Defender model privacy: 0.0
Privacy variance: 0.0
Privacy error: 0.0
